In [ ]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
import re

In [50]:
url='https://www.marmiton.org/recettes/recette_tacos-mexicains_34389.aspx'

In [51]:
r.get(url)

<Response [200]>

In [52]:
soup=BeautifulSoup(r.get(url).content)

In [ ]:
list_of selectors=['h1','div.recipe-author__author>span','div.recipe-infos-users__rate__stars>span','ul.mrtn-tags-list>li','span.recipe-infos-users__value','div.recipe-infos,div.recipe-infos__timmings__detail','div.recipe-infos__timmings__cooking,div.recipe-infos__timmings__preparation']

In [53]:
#Recipe Name
soup.select('h1')[0].text

'Tacos mexicains'

In [54]:
#Author name
soup.select('div.recipe-author__author>span')[0].text.strip('\n ')

'noor_16706694'

In [76]:
#recipe note
soup.select('div.recipe-infos-users__rate__stars>span')[0].text
[i.text for i in soup.select('div.recipe-infos-users__rate__stars>span')]

['4.6/5']

In [56]:
#tags ('entrée, 'plat unique','plats mexicains)
[i.text for i in soup.select('ul.mrtn-tags-list>li')]

['Entrée', 'Plat unique', 'Plats mexicains']

In [64]:
#Nb likes
soup.select('span.recipe-infos-users__value')[1].text

'15k'

In [57]:
#Nb of portions
[i.replace('\n',' ') for i in soup.select('div.recipe-infos,div.recipe-infos__timmings__detail')[0].text.split('\n\n') if len(i)>1]

['temps 55 min', ' personnes 4', ' très facile', 'bon marché']

In [71]:
#cooking time  only
[re.sub('\t|\n|\xa0','',i.text) for i in soup.select('div.recipe-infos__timmings__cooking')]


['Cuisson:25 min']

In [72]:
#Preparation time 
[re.sub('\t|\n|\xa0','',i.text) for i in soup.select('div.recipe-infos__timmings__preparation')]


['Préparation: 30 min']

In [66]:
#total time
soup.select('div.recipe-infos__timmings__total-time ')[0].text

'\nTemps Total : 55 min\t\t\t\t\t\t\t\t'

In [69]:
#nb people
soup.select('span.title-2.recipe-infos__quantity__value')[0].text

'4'

In [ ]:
#Ingredients
[i.text.strip().replace('\n','') for i in soup.select('li.recipe-ingredients__list__item')]

In [81]:
# average cost
[i.text.strip('\n') for i in soup.select('div.recipe-infos__budget')]
        

['bon marché']

In [85]:
# difficulty
[i.text.strip('\n') for i in soup.select('div.recipe-infos__level')]

['très facile']

In [ ]:
#Ustensiles
[re.sub('\n\n|\t*',' ',i.text) for i in soup.select('ul.recipe-utensils__list')]

In [ ]:
[i.text for i in soup.select('ul.recipe-utensils__list')]

In [75]:
# recipe
re.sub('\t|\n','',soup.select('ol.recipe-preparation__list')[0].text)
[i.text for i in soup.select('ol.recipe-preparation__list')]

["\n\n\t\t\tA la poêle, faire dorer l'oignon émincé dans un peu d'huile d'olive. \t\t\n\n\t\t\tRajouter la viande, assaisonner et laisser cuire 5 min. \t\t\n\n\t\t\tLaver les feuilles de laitue. \t\t\n\n\t\t\tCouper les tomates et le poivron en petits dés. \t\t\n\n\t\t\tIncorporer le tout à la poêlée avec le coulis de tomate, et poursuivre la cuisson pendant 5 min. \t\t\n\n\t\t\tEgoutter les haricots rouges et les ajouter 2 min avant la fin de cuisson. \t\t\n\n\t\t\tHors du feu, ajuster l'assaisonnement et saupoudrer généreusement de cumin; on peut aussi rajouter quelques gouttes de Tabasco. \t\t\n\n\t\t\tGarnir les tortillas de préparation et les refermer en les roulant comme des crêpes. Disposer 1 feuille de laitue sur chaque tacos avant de servir.\t\t\n"]

In [ ]:
url1='https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=50&page=4'

In [ ]:
soup1=BeautifulSoup(r.get(url1).content)

In [ ]:
[i['href'] for i in soup1.select('a.recipe-card-link')]


# Code aggregation

In [ ]:
# 14 recipes per page with page from no figure then 2 to 833
#https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=10004&page=833

In [39]:
#liste of links for the pages - start with 5pages to test before launching 833 pages...
# pb le site change de pattern de pagination 
def List_urls(NbPages):
    urls=[]
    for i in range(0,NbPages):
        url=f'https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start={4+14*i}&page={i+1}'
        urls.append(url)
        i+=1
    return urls

page_urls=List_urls(1)    
print(page_urls)

['https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=4&page=1']


In [40]:
#For one given page look for the list of recipes (links)
def find_recipe_card_link(page_urls):
    recipes_of_all_pages=[]
    for url in page_urls:
        soup=BeautifulSoup(r.get(url).content)
        # list of recipes url which are in the page
        recipes_of_all_pages.extend([f'https://www.marmiton.org'+i['href'] for i in soup.select('a.recipe-card-link')])
    return recipes_of_all_pages
        
recipes_of_all_pages=find_recipe_card_link(page_urls) 
recipes_of_all_pages

['https://www.marmiton.org/recettes/recette_original-american-cookies-de-mike_39907.aspx',
 'https://www.marmiton.org/recettes/recette_blanquette-de-veau-facile_19219.aspx',
 'https://www.marmiton.org/recettes/recette_le-crumble-aux-pommes-du-chat-qui-tousse_10534.aspx',
 'https://www.marmiton.org/recettes/recette_gateau-au-yaourt_12719.aspx',
 'https://www.marmiton.org/recettes/recette_veritable-moelleux-au-chocolat_12825.aspx',
 'https://www.marmiton.org/recettes/recette_les-timbales-de-jeanne-saumon-a-la-mousse-de-courgettes-au-micro-ondes_21864.aspx',
 'https://www.marmiton.org/recettes/recette_gateau-aux-pommes-facile_13493.aspx',
 'https://www.marmiton.org/recettes/recette_meringue-pour-les-nuls_36610.aspx',
 'https://www.marmiton.org/recettes/recette_pancakes_15299.aspx',
 'https://www.marmiton.org/recettes/recette_cookie-cup-glace-vanille-et-pepites-de-chocolat_372241.aspx',
 'https://www.marmiton.org/recettes/recette_filet-mignon-en-croute_14407.aspx',
 'https://www.marmiton.o

In [92]:
def Collect_data(recipes_of_page):
    
    
    #recipe=dict() #dictionnary , 1 row of the final dataframe (Recipe name: [list of info])  recipe={'':[]}
    list_of_recipes=[]
    recipe=[]    
    for url in recipes_of_page:

        soup=BeautifulSoup(r.get(url).content)
        # Add Recipe Name
        recipe.append([i.text for i in soup.select('h1')])
        
        # add author name
        recipe.append([i.text.strip('\n ') for i in soup.select('div.recipe-author__author>span')]) 
        
        #add note
        recipe.append([i.text for i in soup.select('div.recipe-infos-users__rate__stars>span')]) 
        
        #add tags
        recipe.append([i.text for i in soup.select('ul.mrtn-tags-list>li')])
        
        # nb likes
        recipe.append(soup.select('span.recipe-infos-users__value')[1].text)
        
        #nb portions
        recipe.append([i.text.strip('\n') for i in soup.select('span.title-2.recipe-infos__quantity__value')])
        
        # difficulty
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__level')])
        
        #add total cooking time
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__total-time ')])
        
        # preparation time
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__preparation')])
            
        # cooking only time
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__cooking')])

        # average cost
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__budget')])
        
        # add url
        recipe.append(url)
        
        #ingredients
        recipe.extend([i.text.strip('\n') for i in soup.select('li.recipe-ingredients__list__item')])
        
        #recipe (text)
        recipe.append([i.text.strip('\n') for i in soup.select('ol.recipe-preparation__list')])
        #check
        print(recipe)
        
        list_of_recipes.append(recipe)
        recipe=[]
    return list_of_recipes

Collect_data(recipes_of_all_pages)

[['Original American Cookies de Mike'], ['nathan_15813882'], ['4.7/5'], ['Confiserie', 'Végétarien', 'Rapide', 'Petits gâteaux', 'Gâteau', 'Cookie', 'Recette américaine', '\n\t\t\t\t\t\tPréparation : 3 min\t\t\t\t\t', '\n\t\t\t\t\t\tPréparation : 1 min\t\t\t\t\t', '\n\t\t\t\t\t\tPréparation : 2 min\t\t\t\t\t', '\n\t\t\t\t\t\tPréparation : 2 min\t\t\t\t\t', '\n\t\t\t\t\t\tPréparation : 3 min\t\t\t\t\t', '\n\t\t\t\t\t\tPréparation : 1 min\t\t\t\t\t', '\n\t\t\t\t\t\tPréparation : 6 min\t\t\t\t\t', '\n\t\t\t\t\t\tCuisson : 9 min\t\t\t\t\t', '\n\t\t\t\t\t\tCuisson : 1 min\t\t\t\t\t'], '121k', ['15'], ['très facile'], ['Temps Total : 15 min\t\t\t\t\t\t\t\t'], ['Préparation\xa0: 5 min\t\t\t\t\t\t\t\t\t\t'], ['Cuisson\xa0:\n\n\t\t\t\t\t\t\t\t\t\t\t\t10 min\t\t\t\t\t\t\t\t\t\t\t'], ['bon marché'], 'https://www.marmiton.org/recettes/recette_original-american-cookies-de-mike_39907.aspx', '250\n\n\n\ng de farine  ', '90\n\n\n\ng de sucre de canne roux (ou cassonade) ', '1\n\n\n\nsachet de sucre va

[['Gâteau aux pommes facile'], ['Anonyme'], ['4.7/5'], ['Dessert', 'Végétarien', 'Gâteau', 'Gâteau aux pommes', "Dessert d'enfance"], '89k', ['1'], ['très facile'], ['Temps Total : 57 min\t\t\t\t\t\t\t\t'], ['Préparation\xa0: 12 min\t\t\t\t\t\t\t\t\t\t'], ['Cuisson\xa0:\n\n\t\t\t\t\t\t\t\t\t\t\t\t45 min\t\t\t\t\t\t\t\t\t\t\t'], ['bon marché'], 'https://www.marmiton.org/recettes/recette_gateau-aux-pommes-facile_13493.aspx', '120\n\n\n\ng de sucre semoule  ', '1\n\n\n\npaquet de sucre vanillé  ', '3\n\n\n\n oeufs  ', '1/2\n\n\n\npaquet de levure chimique  ', '125\n\n\n\ng de farine  ', "1/2\n\n\n\nverre d'huile de colza  ", '3\n\n\n\n pommes  ', ['\t\t\tPréchauffer le four à 160°C (thermostat 5-6).\nMélanger tous les ingrédients de la pâte.\t\t\n\n\t\t\tAjouter les pommes préalablement coupées en cube.\t\t\n\n\t\t\tMélanger à nouveau à la spatule.\t\t\n\n\t\t\tVerser la pâte à gâteau dans un moule beurré et cuire pendant 40 à 50 minutes.\t\t\n\n\t\t\tDéguster !\t\t']]
[['Meringue pour le

[['Fondant au chocolat'], ['Anonyme'], ['4.6/5'], ['Dessert', 'Végétarien', 'Gâteau au chocolat', 'Gâteau', 'Fondant au chocolat', 'Les classiques'], '109k', ['8'], ['très facile'], ['Temps Total : 35 min\t\t\t\t\t\t\t\t'], ['Préparation\xa0: 15 min\t\t\t\t\t\t\t\t\t\t'], ['Cuisson\xa0:\n\n\t\t\t\t\t\t\t\t\t\t\t\t20 min\t\t\t\t\t\t\t\t\t\t\t'], ['bon marché'], 'https://www.marmiton.org/recettes/recette_fondant-au-chocolat_15025.aspx', '200\n\n\n\ng de chocolat à cuire ', '100\n\n\n\ng de beurre doux  ', '100\n\n\n\ng de sucre semoule  ', '5\n\n\n\n oeufs  ', '4\n\n\n\ncuillères à soupe de farine  ', ['\t\t\tPréchauffer le four à 180°C (thermostat 6).\nFaire fondre le chocolat et le beurre au bain-marie à feu doux, ou au micro-ondes sur le programme "décongélation".\t\t\n\n\t\t\tPendant ce temps, séparer les jaunes des blancs d\'oeuf.\t\t\n\n\t\t\tMonter les blancs en neige ferme. Réserver. \t\t\n\n\t\t\tQuand le mélange chocolat-beurre est bien fondu, ajouter les jaunes d’oeufs et foue

[[['Original American Cookies de Mike'],
  ['nathan_15813882'],
  ['4.7/5'],
  ['Confiserie',
   'Végétarien',
   'Rapide',
   'Petits gâteaux',
   'Gâteau',
   'Cookie',
   'Recette américaine',
   '\n\t\t\t\t\t\tPréparation : 3 min\t\t\t\t\t',
   '\n\t\t\t\t\t\tPréparation : 1 min\t\t\t\t\t',
   '\n\t\t\t\t\t\tPréparation : 2 min\t\t\t\t\t',
   '\n\t\t\t\t\t\tPréparation : 2 min\t\t\t\t\t',
   '\n\t\t\t\t\t\tPréparation : 3 min\t\t\t\t\t',
   '\n\t\t\t\t\t\tPréparation : 1 min\t\t\t\t\t',
   '\n\t\t\t\t\t\tPréparation : 6 min\t\t\t\t\t',
   '\n\t\t\t\t\t\tCuisson : 9 min\t\t\t\t\t',
   '\n\t\t\t\t\t\tCuisson : 1 min\t\t\t\t\t'],
  '121k',
  ['15'],
  ['très facile'],
  ['Temps Total : 15 min\t\t\t\t\t\t\t\t'],
  ['Préparation\xa0: 5 min\t\t\t\t\t\t\t\t\t\t'],
  ['Cuisson\xa0:\n\n\t\t\t\t\t\t\t\t\t\t\t\t10 min\t\t\t\t\t\t\t\t\t\t\t'],
  ['bon marché'],
  'https://www.marmiton.org/recettes/recette_original-american-cookies-de-mike_39907.aspx',
  '250\n\n\n\ng de farine  ',
  '90\n\n\n\

In [47]:
url='https://www.marmiton.org/recettes/recette_original-american-cookies-de-mike_39907.aspx'

In [48]:
soup=BeautifulSoup(r.get(url).content) 


In [30]:
a=soup.select('div.recipe-author__author>span')[0].text.strip('\n ')
a
toto=[]
toto.append(a)
print(toto)

['nathan_15813882']


In [22]:
#soup.select('h1','div.recipe-author__author>span','div.recipe-infos-users__rate__stars>span','ul.mrtn-tags-list>li','div.recipe-infos,div.recipe-infos__timmings__detail','span.recipe-infos-users__value','div.recipe-infos__timmings__cooking,div.recipe-infos__timmings__preparation','li.recipe-ingredients__list__item','ol.recipe-preparation__list')

In [ ]:
def create_dataFrame(recipe):
  pd.DataFrame(recipe)

In [ ]:
#'d4 = {}' 'for d in (d1, d2, d3): d4.update(d)'

In [ ]:
# Error Handling function

In [ ]:
# Launch function terminal

In [ ]:
url_page='https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=18&page=2'
soup=BeautifulSoup(r.get(url_page).content)
[f'https://www.marmiton.org'+i['href'] for i in soup.select('a.recipe-card-link')]

In [ ]:
https://www.marmiton.org/recettes/recette_chouquettes_30622.aspx

In [ ]:
#